In [1]:
import conllu_batch_generator as cbg

In [2]:
cr = cbg.ConlluReader("filtered_recipes.conllu")

In [3]:
next(iter(cr))[0]

TokenList<Dissolve, Jello, in, boiling, water, .>

In [4]:
def word2features(sent, i):
    word = sent[i]['form']
    postag = sent[i]['upostag']
    features = [
        'bias',
        #'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1]['form']
        postag1 = sent[i-1]['upostag']
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
        if i > 1:
            word1 = sent[i-2]['form']
            postag1 = sent[i-2]['upostag']
            features.extend([
                '-2:word.lower=' + word1.lower(),
                '-2:word.istitle=%s' % word1.istitle(),
                '-2:word.isupper=%s' % word1.isupper(),
                '-2:postag=' + postag1,
                '-2:postag[:2]=' + postag1[:2],
            ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1]['form']
        postag1 = sent[i+1]['upostag']
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
        if i < len(sent)-2:
            word1 = sent[i+1]['form']
            postag1 = sent[i+1]['upostag']
            features.extend([
                '+2:word.lower=' + word1.lower(),
                '+2:word.istitle=%s' % word1.istitle(),
                '+2:word.isupper=%s' % word1.isupper(),
                '+2:postag=' + postag1,
                '+2:postag[:2]=' + postag1[:2],
            ])
    else:
        features.append('EOS')
                
    return features

In [5]:
def sent2labels(sent):
    labels = []
    for token in sent:
        if token['misc'] is not None and 'food_type' in token['misc']:
            labels.append(token['misc']['food_type'])
        else:
            labels.append("0")
    return labels

In [6]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [7]:
def sent2tokens(sent):
    return [token['form'] for token in sent]

In [8]:
def feature2tokens(sent):
    return [t[1].split("=")[1] for t in sent]

In [9]:
def conllu2tokens(sent):
    return [t['form'] for t in sent]

* create test dataset:

In [10]:
# read 50000 samples:

In [11]:
n_train = 50000
n_test = 1000

X_train = []
Y_train = []
t_train = []

X_test = []
Y_test = []
t_test = []


i = 0
for sample in cr:
    if len(sample) == 0:
        continue
    i += 1
    if i < n_train:
        X_train.append(sent2features(sample[0]))
        Y_train.append(sent2labels(sample[0]))
        t_train.append(conllu2tokens(sample[0]))
    else:
        X_test.append(sent2features(sample[0]))
        Y_test.append(sent2labels(sample[0]))
        t_test.append(conllu2tokens(sample[0]))
    
    if i >= n_train + n_test:
        break



In [12]:
len(X_train)

47538

* train with crfsuite

In [13]:
import pycrfsuite

In [14]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, Y_train):
    trainer.append(xseq, yseq)

In [15]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    #'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [16]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [17]:
trainer.train('test.crfsuite')

In [18]:
trainer.logparser.last_iteration


{'num': 830,
 'scores': {},
 'loss': 41171.669638,
 'feature_norm': 126.341894,
 'error_norm': 85.690855,
 'active_features': 6055,
 'linesearch_trials': 2,
 'linesearch_step': 0.5,
 'time': 0.724}

* test:

In [19]:
tagger = pycrfsuite.Tagger()
tagger.open('test.crfsuite')

In [20]:
for i in range(100,130):
    print(' '.join(t_test[i]))
    #print(' '.join(feature2tokens(X_test[i])), end='\n\n')
    print("Predicted:", ' '.join(tagger.tag(X_test[i])))
    print("Correct:  ", ' '.join(Y_test[i]))
    
    print("\n")

IndexError: list index out of range